
# Shared rides with pricing - Delft 

 ### Choice Function (Deterministic):
 `pool_price.py`
 * Pickup Distance: distance from driver initial position to the first pickup point
 * Travel Distance: distance from driver's initial position to the drop off point of the last passenger
 * Operating Cost: This include all the expenses
 * Profit: Driver revenue to serve the request
            
 
  ### KPI:
   
   * Profit of Individual driver
   * Profit of all the drivers
   * No.of rejected rides
   * U - PAX (Utility) 
  
   ### TBD- Choice Function (Probablistic):
  
  * choice logic to be applied inside `pool_price.py` 
  * P(R)= exp(beta * Profit_R)/ sum_all the rides( exp(beta * Profit_R)
 



-------------------------------------------------------------------------------------------------------

# Pricing and Driver Earnings for a Two-Sided Mobility Platform: A Case of Amsterdam, the Netherlands

or 

# The Effects of Profit-Based Pricing on Driver Earnings and Performance of Two-Sided Mobility Platforms

# Abstract  

In this paper, we investigate how the  pricing of ride-pooling affects driver earnings. We also examine how profit-based setting affects these performance indicators. To this end, we applied a matching algorithm  to the case of ride-pooling and give a choice set to the driver for the case of Amsterdam, the Netherlands. For our simulation, we utilize an agent-based simulator reproducing the transport systems for two-sided mobility platforms (like Uber and Lyft) and applied three state-of-the-art pricing strategies such as <strong>profit maximization</strong>,  <strong>solo ride-hailing</strong>, and <strong>nearest pickup ride-pooling</strong>. We find that the profit maximization pricing strategy outperforms the other and traveler utility can be further improved by $\%X$ while reducing the total cost to serve the pooled rides. While offering a discount for profit maximization travel time is significantly higher $\%X$  than for private rides. 

-------------------------------------------------------------------------------------------------------

## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo ride-hailing</strong>

<strong>3. Nearest pickup ride-pooling</strong>

## Load ExMAS and MaaSSim


In [40]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Delft, Netherlands

In [41]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 300 # reuqests (and passengers)
params.nV = 30 # vehicles



## Parameters for ExMAS

In [42]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.5
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



In [43]:
inData = ExMAS.main(inData, params.shareability, plot=False) # create shareability graph (ExMAS) 

23-05-23 20:40:28-INFO-Initializing pairwise trip shareability between 300 and 300 trips.
23-05-23 20:40:28-INFO-creating combinations
23-05-23 20:40:29-INFO-89700	 nR*(nR-1)
23-05-23 20:40:33-INFO-Reduction of feasible pairs by 99.36%
23-05-23 20:40:33-INFO-Degree 2 	Completed
23-05-23 20:40:33-INFO-trips to extend at degree 2 : 1402
23-05-23 20:40:36-INFO-At degree 2 feasible extensions found out of 290 searched
23-05-23 20:40:36-INFO-Degree 3 	Completed
23-05-23 20:40:36-INFO-trips to extend at degree 3 : 290
23-05-23 20:40:36-INFO-At degree 3 feasible extensions found out of 37 searched
23-05-23 20:40:36-INFO-Degree 4 	Completed
23-05-23 20:40:36-INFO-Max degree reached 4
23-05-23 20:40:36-INFO-Trips still possible to extend at degree 4 : 37


In [44]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

23-05-23 20:40:38-INFO-Matching 300 trips to 2029 rides in order to min u_veh
23-05-23 20:40:39-INFO-Problem solution: Optimal. 
Total costs for single trips:         79,454 
reduced by matching to:               58,875


In [45]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],2.090500,113,1,[2.0905],"[0, 113]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 4217813217, 44835265]","[None, 0, 0]",node time req_id od 0 ...,113,1.13,1.695000,0.339000,1.3560
1,[1],7.352000,397,1,[7.352],"[5, 397]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44759522, 1530303700]","[None, 1, 1]",node time req_id od 0 ...,397,3.97,5.962500,1.192500,4.7700
2,[2],4.287000,231,1,[4.287],"[9, 231]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 4883670378, 1435362502]","[None, 2, 2]",node time req_id od 0 ...,231,2.31,3.478500,0.695700,2.7828
3,[3],4.077500,220,1,[4.077500000000001],"[39, 220]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1371031524, 44813102]","[None, 3, 3]",node time req_id od 0 ...,220,2.20,3.307500,0.661500,2.6460
4,[4],2.875000,155,1,[2.875],"[60, 155]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44814383, 311132084]","[None, 4, 4]",node time req_id od 0 N...,155,1.55,2.332500,0.466500,1.8660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,"[50, 58, 57, 60]",14.729925,575,40,"[4.4605500000000005, 3.0061999999999998, 3.088...","[432.5, 99, 34, 148, 13, 13, 22, 246]","[50, 58, 57, 60]","[58, 50, 57, 60]",4,2024,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 3510592637, 4301397894, 44825205, 13930...","[None, 50, 58, 57, 60, 58, 50, 57, 60]",node time req_id od 0 ...,575,5.75,9.174375,1.834875,7.3395
2025,"[50, 49, 57, 58]",13.638650,332,40,"[4.3900250000000005, 3.25845, 3.07725000000000...","[423.0, 0, 117, 34, 77, 74, 22, 8]","[50, 49, 57, 58]","[49, 50, 57, 58]",4,2025,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 3510592637, 3510592637, 44825205, 43013...","[None, 50, 49, 57, 58, 49, 50, 57, 58]",node time req_id od 0 ...,332,3.32,8.597250,1.719450,6.8778
2026,"[50, 49, 58, 57]",13.511250,301,40,"[4.339975000000001, 3.2084, 2.9993749999999997...","[423.0, 0, 99, 34, 73, 74, 13, 8]","[50, 49, 58, 57]","[49, 50, 58, 57]",4,2026,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 3510592637, 3510592637, 4301397894, 448...","[None, 50, 49, 58, 57, 49, 50, 58, 57]",node time req_id od 0 ...,301,3.01,8.597250,1.719450,6.8778
2027,"[50, 49, 29, 58]",16.926850,528,42,"[4.221675, 2.9991000000000003, 6.6748499999999...","[423.0, 0, 51, 68, 77, 74, 13, 245]","[50, 49, 29, 58]","[49, 50, 58, 29]",4,2027,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 3510592637, 3510592637, 44847014, 43013...","[None, 50, 49, 29, 58, 49, 50, 58, 29]",node time req_id od 0 ...,528,5.28,10.775250,2.155050,8.6202


In [46]:
params.kpi = 1

In [47]:
sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate

23-05-23 20:41:26-INFO-Matching 300 trips to 2029 rides in order to min u_veh
23-05-23 20:41:27-INFO-Problem solution: Optimal. 
Total costs for single trips:         79,454 
reduced by matching to:               58,875
0.5
23-05-23 20:42:11-CRITICAL-this is request 0 with [0, 680, 975, 1322] available rides.
23-05-23 20:42:11-CRITICAL-ride 0 available [1]
23-05-23 20:42:11-CRITICAL-ride 680 available [1, 0]
23-05-23 20:42:11-CRITICAL-ride 975 available [1, 0]
23-05-23 20:42:11-CRITICAL-ride 1322 available [1, 0]
23-05-23 20:42:11-CRITICAL-this is reuqest 0 with [0, 680, 975, 1322] still available rides.
Profit Maximization
23-05-23 20:42:11-CRITICAL-vehicle 1 has 4 choices
0.5
23-05-23 20:42:12-CRITICAL-this is request 1 with [1, 500, 575, 678, 688, 712, 1020, 1500, 1501, 1503, 1504, 1505, 1506, 1507, 1511] available rides.
23-05-23 20:42:12-CRITICAL-ride 1 available [1]
23-05-23 20:42:12-CRITICAL-ride 500 available [1, 0]
23-05-23 20:42:12-CRITICAL-ride 575 available [1, 0]
23-05-23 

In [48]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,REPOSITIONED,DECIDES_NOT_TO_DRIVE,ENDS_SHIFT,TRAVEL,WAIT,CRUISE,OPERATIONS,IDLE,PAX_KM,REVENUE
veh,,,,,,,,,,,,,,,,,,,,,
1,12.0,0.0,False,0,0,169.0,60.0,0,270.0,0,...,0,0,6847.0,1846.0,0.0,1247.0,690.0,3064.0,27.570,10.5210
2,0.0,0.0,False,0,0,418.0,0.0,0,0.0,0,...,0,0,10381.0,0.0,0.0,0.0,0.0,10381.0,0.000,0.0000
3,8.0,0.0,False,0,0,577.0,45.0,0,189.0,0,...,0,0,6964.0,1469.0,0.0,1315.0,474.0,3706.0,20.991,7.3197
4,8.0,0.0,False,0,0,247.0,30.0,0,440.0,0,...,0,0,0.0,1422.0,3600.0,637.0,685.0,-6344.0,31.667,6.0417
5,12.0,0.0,False,0,0,544.0,45.0,0,279.0,0,...,0,0,6045.0,1632.0,0.0,1894.0,684.0,1835.0,22.318,1.2540
6,8.0,0.0,False,0,0,318.0,30.0,0,184.0,0,...,0,0,6807.0,1762.0,0.0,1458.0,454.0,3133.0,27.245,3.3072
7,10.0,0.0,False,0,0,268.0,15.0,0,500.0,0,...,0,0,6570.0,2399.0,0.0,747.0,815.0,2609.0,33.096,6.5403
8,12.0,0.0,False,0,0,216.0,100.0,0,339.0,0,...,0,0,6879.0,1785.0,0.0,1120.0,799.0,3175.0,27.015,9.0549
9,10.0,0.0,False,0,0,151.0,30.0,0,650.0,0,...,0,0,6279.0,2106.0,0.0,1283.0,980.0,1910.0,37.906,3.5760


In [49]:
sim.res[0].pax_exp

,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,LOSES_PATIENCE,MEETS_DRIVER_AT_PICKUP,RECEIVES_OFFER,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,TRAVEL,WAIT,OPERATIONS,LOST_PATIENCE
pax,,,,,,,,,,,,,,,,,,
0,15.0,200.0,20.0,30.0,0.0,362.0,26.0,0.0,10.0,0,0,0,0,0,200.0,388.0,55.0,False
1,15.0,436.0,20.0,30.0,0.0,34.0,221.0,5.0,10.0,0,0,0,0,0,436.0,255.0,55.0,False
2,15.0,231.0,20.0,30.0,0.0,29.0,0.0,9.0,10.0,0,0,0,0,0,231.0,29.0,55.0,False
3,15.0,220.0,20.0,30.0,0.0,22.0,0.0,39.0,10.0,0,0,0,0,0,220.0,22.0,55.0,False
4,15.0,155.0,20.0,30.0,0.0,26.0,0.0,60.0,10.0,0,0,0,0,0,155.0,26.0,55.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.0,575.0,20.0,30.0,0.0,12.0,67.0,3525.0,10.0,0,0,0,0,0,575.0,79.0,40.0,False
296,0.0,434.0,20.0,30.0,0.0,80.0,87.0,3529.0,10.0,0,0,0,0,0,434.0,167.0,40.0,False
297,0.0,328.0,20.0,30.0,0.0,42.0,89.0,3545.0,10.0,0,0,0,0,0,328.0,131.0,40.0,False


In [50]:
Fulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[False]
unFulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[True]
service_rate = Fulfilled_requests/(Fulfilled_requests+unFulfilled_requests)
print('service_rate = ', service_rate*100, '%')

service_rate =  97.0 %


In [51]:
#sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/veh{}.csv')
#sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/pax{}.csv')